In [1]:
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
from processor.processor import Processor as p
from modeler.modeler import Modeler as m
from database.market import Market
from strats.speculation import Speculation
import numpy as np
import pickle

In [2]:
start_year = datetime.now().year
training_year = 4

In [3]:
market = Market()

In [4]:
strat_class = Speculation(training_year)
strat_class.pull_included_columns()
strat_class.pull_factors()

In [5]:
market.connect()
sp5 = market.retrieve("sp500")
deactivated_tickers = market.retrieve("deactivated_tickers")
unmodeled_tickers = market.retrieve("unmodeled_tickers")
market.disconnect()

In [6]:
tickers = [x for x in sp5["Symbol"] if x not in list(deactivated_tickers["ticker"]) and x not in list(unmodeled_tickers["tickers"])]

In [7]:
market.connect()
data = []
for ticker in tqdm(tickers):
    try:
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        complete = strat_class.transform_live(prices,ticker)
        data.append(complete[complete["year"]==start_year].tail(1))
    except Exception as e:
        print(str(e))
        continue
final_data = pd.concat(data)
market.disconnect()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 403/403 [00:34<00:00, 11.53it/s]


In [8]:
final_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,d1,d2,d3,year,week,ticker
1181,140.380000,148.358191,149.767273,145.167778,139.468000,132.42400,129.204000,129.455000,130.160000,131.222500,133.346000,144.066000,147.465000,146.494000,-0.053777,4.715756,-4.636043,2022,34,MMM
1181,60.741111,63.872000,62.103000,61.230000,60.388853,57.49604,55.564906,57.717524,55.055246,53.728338,54.643138,60.448486,60.362879,58.951359,-0.049018,-2.720846,-3.726678,2022,34,AOS
1181,105.720000,110.900000,109.929000,109.426667,109.356000,109.01400,107.299697,108.701699,108.322385,105.906626,103.191199,113.561093,116.138633,114.202242,-0.046709,-6.287998,-7.804114,2022,34,ABT
1181,265.584444,283.372000,290.934000,294.203333,285.868000,275.01200,260.476000,261.077500,252.484000,243.127500,231.420000,260.246000,262.735000,256.002000,-0.062771,1.415000,0.056757,2022,34,ABMD
1102,307.065556,318.862000,314.894000,307.580000,296.388000,281.27400,271.790395,276.409889,281.901318,287.557659,274.838490,298.005922,301.096906,286.586620,-0.036995,-3.935899,7.372830,2022,34,ACN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036,62.427778,66.154000,66.833000,65.743333,62.780000,60.69000,54.356000,56.987500,58.072000,54.502500,56.882000,66.630000,65.782500,61.908000,-0.056326,4.544135,-3.817253,2022,34,WYNN
1181,75.667778,76.912000,75.015000,73.907778,71.062000,68.27600,69.572000,70.020000,70.584000,66.595000,65.695529,72.612679,74.652584,75.070492,-0.016177,-1.639712,-3.383274,2022,34,XEL
567,96.287778,102.332000,100.766000,95.803333,87.694000,80.83200,76.746000,79.350000,78.482000,76.692500,75.178000,83.834000,84.997500,82.691609,-0.059065,-4.800592,5.858137,2022,34,XYL
1181,114.827702,118.198000,118.141000,120.456667,120.130000,119.78000,116.876000,117.295000,114.754000,112.445000,110.372000,118.366000,119.952500,115.871089,-0.028514,-60.099518,57.628104,2022,34,YUM


In [12]:
strat_class.db.connect()
models = strat_class.db.retrieve("models")
for year in tqdm(range(start_year,start_year+1)):
    try:
        ticker_data = final_data.copy()
        prediction_set = ticker_data[ticker_data["year"]==year].dropna()
        ticker_models = models[(models["year"]==year) & (models["training_year"]==training_year)]
        ticker_models["model"] = [pickle.loads(x) for x in ticker_models["model"]]
        prediction_set = m.predict(ticker_models,prediction_set,strat_class.factors)
        prediction_set = strat_class.prediction_clean(prediction_set)
        strat_class.db.store("recs",prediction_set)
    except Exception as e:
        print(year,str(e))
strat_class.db.disconnect()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 23.80it/s]


In [13]:
prediction_set

,skl_prediction,skl_score,xgb_prediction,xgb_score,cat_prediction,cat_score,year,week,ticker,training_year
1181,140.595996,0.998375,139.128250,0.999207,136.738538,0.93906,2022,35,MMM,4
1181,60.615008,0.998375,60.835354,0.999207,58.319660,0.93906,2022,35,AOS,4
1181,105.787162,0.998375,106.921005,0.999207,99.514937,0.93906,2022,35,ABT,4
1181,266.093395,0.998375,265.368988,0.999207,235.935494,0.93906,2022,35,ABMD,4
1102,307.780715,0.998375,311.521332,0.999207,310.384473,0.93906,2022,35,ACN,4
...,...,...,...,...,...,...,...,...,...,...
1036,61.828646,0.998375,61.093178,0.999207,56.040496,0.93906,2022,35,WYNN,4
1181,75.709283,0.998375,75.825676,0.999207,64.272802,0.93906,2022,35,XEL,4
567,95.630876,0.998375,95.615952,0.999207,81.260847,0.93906,2022,35,XYL,4
1181,115.137371,0.998375,115.539169,0.999207,115.152399,0.93906,2022,35,YUM,4
